# Trading Bot

In [5]:
from datetime import datetime
import os
import time

import schedule

from utilities.charting import plot_candlesticks_RSI_chart
from utilities.getData_YahooFinanace import update_daily_data, update_60m_data
from utilities.indicators import calculate_rsi

In [2]:
# Set Global Variables
tickers = ["RELIANCE.NS","AAPL"]

**Download data and Calculate RSI**

### Scheduled Runs

In [6]:
# Clear any old jobs
schedule.clear()

# Update Daily and Intraday data
try:
    # Schedule daily data update
    schedule.every().day.at("10:00").do(update_daily_data, tickers)
    schedule.every().day.at("14:00").do(update_daily_data, tickers)
    # Update 60m data
    schedule.every(5).minutes.do(update_60m_data, tickers)

    while True:
        schedule.run_pending()
        time.sleep(60)  # Sleep for 60 seconds
except KeyboardInterrupt:
    print("\n🛑 Scheduler stopped by user. Goodbye!")

[*********************100%***********************]  1 of 1 completed

⬇️ Downloading RELIANCE.NS from 2025-04-30 09:15:00 to 2025-04-30 10:15:00...
✅ RELIANCE.NS is already up to date. No new rows added.
⬇️ Downloading AAPL from 2025-04-29 15:30:00 to 2025-04-30 10:15:00...



[*********************100%***********************]  1 of 1 completed


✅ AAPL is already up to date. No new rows added.


[*********************100%***********************]  1 of 1 completed

...

✅ AAPL is already up to date. No new rows added.


[*********************100%***********************]  1 of 1 completed

⬇️ Downloading RELIANCE.NS from 2025-04-30 09:15:00 to 2025-04-30 11:15:00...



[*********************100%***********************]  1 of 1 completed


✅ RELIANCE.NS updated with 1 new rows at 11:17:57.615170.
⬇️ Downloading AAPL from 2025-04-29 15:30:00 to 2025-04-30 11:15:00...
✅ AAPL is already up to date. No new rows added.

🛑 Scheduler stopped by user. Goodbye!


### Manual Runs

In [4]:
# Update the Daily data
update_daily_data(tickers)

⬇️ Downloading RELIANCE.NS from 2025-04-29 to 2025-04-29...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

✅ RELIANCE.NS updated with 1 new rows at 18:24:41.149568.
⬇️ Downloading AAPL from 2025-04-29 to 2025-04-29...
✅ AAPL is already up to date. No new rows to add.


In [3]:
# Update the 60m data
update_60m_data(tickers)

⬇️ Downloading RELIANCE.NS from 2025-04-28 15:15:00 to 2025-04-29 17:15:00...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

✅ RELIANCE.NS updated with 7 new rows.
⬇️ Downloading AAPL from 2025-04-28 15:30:00 to 2025-04-29 17:15:00...
✅ AAPL is already up to date. No new rows added.
